# E-commerce Shipping Analysis
## Identifying Late Deliveries and Seller Performance

### Objective
This notebook analyzes e-commerce data to identify orders where sellers missed the shipping deadline by delivering packages to carriers after the designated limit date set by Olist.

### Business Problem
When sellers deliver packages late to carriers, it causes:
- Customer dissatisfaction
- Delayed deliveries
- Operational inefficiencies
- Potential financial penalties

### Analysis Goals
1. Identify the percentage of late deliveries
2. Find which product categories have the most late deliveries
3. Calculate average delay times
4. Identify sellers with frequent late deliveries
5. Provide actionable insights for operations team

### Datasets Used
- `olist_order_items_dataset.csv`: Order item details and shipping limits
- `olist_orders_dataset.csv`: Order status and delivery timelines
- `olist_products_dataset.csv`: Product category information

### Methodology
We'll use pandas for data manipulation and analysis, comparing `shipping_limit_date` with `order_delivered_carrier_date` to identify late deliveries.

---

## Step 1: Setup and Data Loading
First, let's import necessary libraries and load our datasets.

In [ ]:
import pandas as pd
import zipfile
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## Step 2: Extract and Load Data
To Do: Extract the zip file and load the three main datasets

In [1]:
# To Do 1: Extract the zip file containing the e-commerce data
# Use zipfile to extract all files from 'data/ecommerce_zipped_raw_data.zip'
# Hint: Use ZipFile context manager and extractall() method

with zipfile.ZipFile('data/ecommerce_zipped_raw_data.zip', 'r') as zip_ref:
	zip_ref.extractall('data/ecommerce_raw_data')

print("Files extracted successfully!")

# To Do 2: Load the three main datasets we need
# Read these CSV files into pandas DataFrames:
# - olist_order_items_dataset.csv
# - olist_orders_dataset.csv  
# - olist_products_dataset.csv
# Use meaningful variable names: df_items, df_orders, df_products

df_items = pd.read_csv('data/ecommerce_raw_data/olist_order_items_dataset.csv')
df_orders = pd.read_csv('data/ecommerce_raw_data/olist_orders_dataset.csv')
df_products = pd.read_csv('data/ecommerce_raw_data/olist_products_dataset.csv')

print("Datasets loaded successfully!")
print(f"Items shape: {df_items.shape}")
print(f"Orders shape: {df_orders.shape}")
print(f"Products shape: {df_products.shape}")

NameError: name 'zipfile' is not defined

## Step 3: Data Exploration
To Do: Explore the structure of each dataset to understand what we're working with

In [ ]:
# To Do 3: Explore the data structure
# Use .head(), .info(), and .describe() to understand each dataset
# Look at the columns and data types

print("\n--- Items Dataset Preview ---")
print(df_items.head())
print("\n--- Items Dataset Info ---")
print(df_items.info())
print("\n--- Items Dataset Description ---")
print(df_items.describe(include='all'))

print("\n--- Orders Dataset Preview ---")
print(df_orders.head())
print("\n--- Orders Dataset Info ---")
print(df_orders.info())
print("\n--- Orders Dataset Description ---")
print(df_orders.describe(include='all'))

print("\n--- Products Dataset Preview ---")
print(df_products.head())
print("\n--- Products Dataset Info ---")
print(df_products.info())
print("\n--- Products Dataset Description ---")
print(df_products.describe(include='all'))

## Step 4: Data Cleaning - Date Conversion
To Do: Convert date columns to proper datetime format for accurate comparisons

In [ ]:
# To Do 4: Convert date columns to datetime format
# Identify date columns in each dataset and convert them using pd.to_datetime()
# Date columns to check: shipping_limit_date, order_purchase_timestamp, 
# order_approved_at, order_delivered_carrier_date, order_delivered_customer_date,
# order_estimated_delivery_date

# Convert date columns in df_items
df_items['shipping_limit_date'] = pd.to_datetime(df_items['shipping_limit_date'])

# Convert date columns in df_orders
date_cols_orders = [
	'order_purchase_timestamp', 
	'order_approved_at', 
	'order_delivered_carrier_date', 
	'order_delivered_customer_date', 
	'order_estimated_delivery_date'
]
for col in date_cols_orders:
	df_orders[col] = pd.to_datetime(df_orders[col])

print("Date columns converted successfully!")
print("\nSample of converted dates from items:")
print(df_items[['shipping_limit_date']].head())
print("\nSample of converted dates from orders:")
print(df_orders[['order_purchase_timestamp', 'order_delivered_carrier_date']].head())

## Step 5: Data Integration - Merging Datasets
To Do: Combine the three datasets to create a comprehensive view of orders, items, and products

In [ ]:
# To Do 5: Merge the datasets
# Combine the three datasets using appropriate merge operations
# Hint: You'll need to merge items with orders on 'order_id', 
# then merge the result with products on 'product_id'
# Keep only the product_category_name from products dataset

# Merge items with orders on 'order_id'
df_merged = pd.merge(df_items, df_orders, on='order_id', how='inner')

# Merge the result with products on 'product_id', keeping only 'product_category_name'
df_merged = pd.merge(
	df_merged,
	df_products[['product_id', 'product_category_name']],
	on='product_id',
	how='left'
)

print(f"Merged dataset shape: {df_merged.shape}")
print("\nMerged dataset columns:")
print(df_merged.columns.tolist())
print("\nFirst few rows of merged data:")
print(df_merged.head())

## Step 6: Core Analysis - Identifying Late Deliveries
To Do: Filter the data to find orders where sellers missed the shipping deadline

In [ ]:
# To Do 6: Identify late deliveries
# Filter the merged dataset to find orders where shipping_limit_date 
# is earlier than order_delivered_carrier_date
# This identifies orders delivered to carrier after the deadline

late_deliveries = df_merged[
	df_merged['order_delivered_carrier_date'] > df_merged['shipping_limit_date']
]

print(f"Total orders: {len(df_merged):,}")
print(f"Late deliveries: {len(late_deliveries):,}")
print(f"Percentage late: {len(late_deliveries)/len(df_merged)*100:.2f}%")

# Display basic statistics about late deliveries
print("\nLate deliveries overview:")
print(f"- Average price: ${late_deliveries['price'].mean():.2f}")
print(f"- Average freight value: ${late_deliveries['freight_value'].mean():.2f}")

## Step 7: Sample Analysis
To Do: Examine sample late deliveries to understand the pattern

In [ ]:
# To Do 7: Display sample of late deliveries
# Show the first 10 late deliveries with key columns:
# order_id, seller_id, shipping_limit_date, order_delivered_carrier_date, product_category_name

# Define the columns to display
sample_columns = [
	'order_id', 
	'seller_id', 
	'shipping_limit_date', 
	'order_delivered_carrier_date', 
	'product_category_name'
]

# Filter late deliveries (if not already defined)
late_deliveries = df_merged[
	df_merged['order_delivered_carrier_date'] > df_merged['shipping_limit_date']
]

print("Sample of late deliveries:")
print(late_deliveries[sample_columns].head(10))

## Step 8: Order Status Analysis
To Do: Analyze the distribution of order statuses to understand overall order flow

In [ ]:
# To Do 8: Analyze by order status
# Group the merged data by order_status and count the number of orders in each status
# This helps understand the distribution of order statuses

order_status_counts = df_merged['order_status'].value_counts()

# Visualize order status distribution
plt.figure(figsize=(10, 6))
order_status_counts.plot(kind='bar', color='skyblue')
plt.title('Distribution of Orders by Status')
plt.xlabel('Order Status')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Step 9: Product Category Analysis
To Do: Identify which product categories have the most late deliveries

In [ ]:
# To Do 9: Analyze late deliveries by product category
# Group late deliveries by product_category_name and count occurrences
# Sort to see which categories have the most late deliveries

late_by_category = late_deliveries['product_category_name'].value_counts()

# Visualize top categories with late deliveries
plt.figure(figsize=(12, 6))
late_by_category.head(15).plot(kind='bar', color='salmon')
plt.title('Top 15 Product Categories with Late Deliveries')
plt.xlabel('Product Category')
plt.ylabel('Number of Late Deliveries')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Step 10: Delay Time Analysis
To Do: Calculate how late deliveries actually are in terms of hours

In [ ]:
# To Do 10: Calculate delay time
# Create a new column 'delay_hours' that calculates how many hours late each delivery was
# Hint: Subtract shipping_limit_date from order_delivered_carrier_date and convert to hours

# Calculate delay in hours
late_deliveries['delay_hours'] = (
    (late_deliveries['order_delivered_carrier_date'] - late_deliveries['shipping_limit_date'])
    .dt.total_seconds() / 3600
)

print(f"Average delay: {late_deliveries['delay_hours'].mean():.2f} hours")
print(f"Maximum delay: {late_deliveries['delay_hours'].max():.2f} hours")
print(f"Minimum delay: {late_deliveries['delay_hours'].min():.2f} hours")
print(f"Median delay: {late_deliveries['delay_hours'].median():.2f} hours")

# Visualize delay time distribution
plt.figure(figsize=(12, 6))
plt.hist(late_deliveries['delay_hours'], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
plt.title('Distribution of Delivery Delay Times')
plt.xlabel('Delay Hours')
plt.ylabel('Frequency')
plt.axvline(
    late_deliveries['delay_hours'].mean(),
    color='red',
    linestyle='--',
    label=f"Mean: {late_deliveries['delay_hours'].mean():.1f} hours"
)
plt.legend()
plt.tight_layout()
plt.show()

## Step 11: Seller Performance Analysis
To Do: Identify sellers who frequently miss shipping deadlines

In [ ]:
# To Do 11: Identify sellers with most late deliveries
# Group late deliveries by seller_id and count occurrences
# Sort to see which sellers have the most late deliveries

# Group late deliveries by seller_id and count occurrences
late_by_seller = late_deliveries['seller_id'].value_counts()

# Analyze seller performance
print(f"\nTotal unique sellers with late deliveries: {len(late_by_seller)}")
print(f"Top seller has {late_by_seller.iloc[0]} late deliveries")
print(f"Average late deliveries per seller: {late_by_seller.mean():.2f}")

## Step 12: Save Results
To Do: Save the analysis results for further investigation and reporting

In [ ]:
# To Do 12: Save results to CSV
# Save the late_deliveries DataFrame to a CSV file for further analysis

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

late_deliveries.to_csv('missed_shipping_limit_orders.csv', index=False)
print("Results saved to 'missed_shipping_limit_orders.csv'")

# Also save the summary statistics
summary_stats = pd.DataFrame({
    'metric': ['Total Orders', 'Late Deliveries', 'Late Delivery Rate', 
               'Average Delay Hours', 'Max Delay Hours', 'Unique Sellers with Late Deliveries'],
    'value': [len(df_merged), len(late_deliveries), 
              len(late_deliveries)/len(df_merged)*100,
              late_deliveries['delay_hours'].mean(),
              late_deliveries['delay_hours'].max(),
              len(late_by_seller)]
})
summary_stats.to_csv('shipping_analysis_summary.csv', index=False)
print("Summary statistics saved to 'shipping_analysis_summary.csv'")

## Step 13: Comprehensive Summary Report
To Do: Generate a final summary report with key findings and insights

In [ ]:
# To Do 13: Create a summary report
# Generate a comprehensive summary of your findings

# Ensure delay_hours column exists
if 'delay_hours' not in late_deliveries.columns:
    late_deliveries['delay_hours'] = (
        (late_deliveries['order_delivered_carrier_date'] - late_deliveries['shipping_limit_date'])
        .dt.total_seconds() / 3600
    )

print("\n" + "="*60)
print("E-COMMERCE SHIPPING ANALYSIS SUMMARY")
print("="*60)
print(f"Total orders analyzed: {len(df_merged):,}")
print(f"Late deliveries found: {len(late_deliveries):,}")
print(f"Late delivery rate: {len(late_deliveries)/len(df_merged)*100:.2f}%")
print(f"Average delay time: {late_deliveries['delay_hours'].mean():.2f} hours")
print(f"Maximum delay time: {late_deliveries['delay_hours'].max():.2f} hours")
print(f"Product category with most late deliveries: {late_by_category.index[0]}")
print(f"Seller with most late deliveries: {late_by_seller.index[0]}")
print(f"\nTop 5 problematic categories:")
for i, (category, count) in enumerate(late_by_category.head().items(), 1):
    print(f"  {i}. {category}: {count} late deliveries")
print("="*60)
print("\nRECOMMENDATIONS:")
print("1. Focus on top 5 product categories for process improvement")
print("2. Work with sellers having frequent late deliveries")
print("3. Implement stricter monitoring for high-value orders")
print("4. Consider revising shipping deadlines for problematic categories")

## Step 14: Bonus - Advanced Visualizations (Optional)
To Do: Create additional visualizations to deepen the analysis

In [ ]:
# To Do 14: Bonus - Visualization (optional)
# Create some advanced visualizations using matplotlib or seaborn

# Ensure 'delay_hours' exists in late_deliveries
if 'delay_hours' not in late_deliveries.columns:
    late_deliveries['delay_hours'] = (
        (late_deliveries['order_delivered_carrier_date'] - late_deliveries['shipping_limit_date'])
        .dt.total_seconds() / 3600
    )

try:
    # 1. Price vs Delay time scatter plot
    plt.figure(figsize=(12, 6))
    plt.scatter(late_deliveries['price'], late_deliveries['delay_hours'], 
               alpha=0.6, color='purple')
    plt.title('Relationship between Order Price and Delay Time')
    plt.xlabel('Order Price ($)')
    plt.ylabel('Delay Hours')
    plt.xlim(0, 1000)  # Limit x-axis for better visualization
    plt.tight_layout()
    plt.show()

    # 2. Delay time by product category (top 10)
    plt.figure(figsize=(14, 7))
    top_categories = late_by_category.head(10).index
    category_delays = late_deliveries[late_deliveries['product_category_name'].isin(top_categories)]
    
    sns.boxplot(data=category_delays, x='product_category_name', y='delay_hours')
    plt.title('Delay Time Distribution by Top Product Categories')
    plt.xlabel('Product Category')
    plt.ylabel('Delay Hours')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print("Advanced visualizations created successfully!")
    
except Exception as e:
    print(f"Error creating advanced visualizations: {e}") Do 14: Bonus - Visualization (optional)
# Create some advanced visualizations using matplotlib or seaborn

# Ensure 'delay_hours' exists in late_deliveries
if 'delay_hours' not in late_deliveries.columns:
    late_deliveries['delay_hours'] = (
        (late_deliveries['order_delivered_carrier_date'] - late_deliveries['shipping_limit_date'])
        .dt.total_seconds() / 3600
    )

try:
    # 1. Price vs Delay time scatter plot
    plt.figure(figsize=(12, 6))
    plt.scatter(late_deliveries['price'], late_deliveries['delay_hours'], 
               alpha=0.6, color='purple')
    plt.title('Relationship between Order Price and Delay Time')
    plt.xlabel('Order Price ($)')
    plt.ylabel('Delay Hours')
    plt.xlim(0, 1000)  # Limit x-axis for better visualization
    plt.tight_layout()
    plt.show()

    # 2. Delay time by product category (top 10)
    plt.figure(figsize=(14, 7))
    top_categories = late_by_category.head(10).index
    category_delays = late_deliveries[late_deliveries['product_category_name'].isin(top_categories)]
    
    sns.boxplot(data=category_delays, x='product_category_name', y='delay_hours')
    plt.title('Delay Time Distribution by Top Product Categories')
    plt.xlabel('Product Category')
    plt.ylabel('Delay Hours')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print("Advanced visualizations created successfully!")
    
except Exception as e:
    print(f"Error creating advanced visualizations: {e}")

# Conclusion

## Key Findings
- **Late Delivery Rate**: X% of orders missed the shipping deadline
- **Problematic Categories**: [Top categories] show the highest incidence of late deliveries
- **Delay Patterns**: Average delay is X hours, with some extreme cases
- **Seller Performance**: [Number] sellers account for the majority of late deliveries

## Next Steps
1. **Operational Improvements**: Focus on top categories and sellers
2. **Process Review**: Analyze why certain categories have more delays
3. **Seller Training**: Work with problematic sellers on process improvement
4. **Monitoring**: Implement real-time monitoring of shipping deadlines

## Files Generated
- `missed_shipping_limit_orders.csv`: Complete list of late deliveries
- `shipping_analysis_summary.csv`: Key metrics and statistics
- Visualizations: Multiple charts showing patterns and distributions

---
*Analysis completed successfully!*